<a href="https://colab.research.google.com/github/romansvintitskyy/QuotePop/blob/main/backend_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.1 MB/s 
     |████████████████████████████████| 5.5 MB 44.0 MB/s 
     |████████████████████████████████| 1.3 MB 39.2 MB/s 
     |████████████████████████████████| 163 kB 57.9 MB/s 
     |████████████████████████████████| 7.6 MB 54.3 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=6362e34e4d877561c0df9ee0979fe4b5b8f24401fed23822e90cd64f219cf175
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [2]:
# Importing libraries

import math
import http.client
import json
from sentence_transformers import SentenceTransformer, util

Part 1: Matching our sentences to available categories and retrieving n-quotes

In [3]:
# Downloading the model
# pre vectorization to speed shit up
model = SentenceTransformer('all-mpnet-base-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [4]:
# Establishing connection with RapidAPI and getting the list of quote categories

conn = http.client.HTTPSConnection("famous-quotes4.p.rapidapi.com")

headers = {
    'X-RapidAPI-Key': "12362c4f69mshf3bc1981e616ac1p1939abjsnde94f542c73e",
    'X-RapidAPI-Host': "famous-quotes4.p.rapidapi.com"
    }

conn.request("GET", "/", headers=headers)

res = conn.getresponse()
data = res.read()
categories = json.loads(data)

print(categories)

['age', 'alone', 'amazing', 'anger', 'anniversary', 'architecture', 'art', 'attitude', 'beauty', 'best', 'birthday', 'business', 'car', 'change', 'communication', 'computers', 'cool', 'courage', 'dad', 'dating', 'death', 'design', 'diet', 'dreams', 'education', 'environmental', 'equality', 'experience', 'failure', 'faith', 'family', 'famous', 'fear', 'finance', 'fitness', 'food', 'forgiveness', 'freedom', 'friendship', 'funny', 'future', 'gardening', 'god', 'good', 'government', 'graduation', 'great', 'happiness', 'health', 'history', 'home', 'hope', 'humor', 'imagination', 'inspirational', 'intelligence', 'jealousy', 'knowledge', 'leadership', 'learning', 'legal', 'life', 'love', 'marriage', 'medical', 'men', 'mom', 'money', 'morning', 'motivational', 'movies', 'movingon', 'music', 'nature', 'parenting', 'patience', 'patriotism', 'peace', 'pet', 'poetry', 'politics', 'positive', 'power', 'relationship', 'religion', 'respect', 'romantic', 'sad', 'science', 'smile', 'society', 'sports',

In [5]:
# import http.client

# conn = http.client.HTTPSConnection("famous-quotes4.p.rapidapi.com")

# headers = {
#     'X-RapidAPI-Key': "12362c4f69mshf3bc1981e616ac1p1939abjsnde94f542c73e",
#     'X-RapidAPI-Host': "famous-quotes4.p.rapidapi.com"
#     }

# conn.request("GET", "/random?category=all&count=100000", headers=headers)

# res = conn.getresponse()
# data = res.read()

# print(data.decode("utf-8"))

In [6]:
# The sentences we'd like to encode
# It's a hard-coded example for the sake of having a quick demo. 
# Later on, this will be replaced directly by the user input.
ourSentences = ["I am so sorry you are going through a hard time. I wish I could be there for you. But don't worry, it will get better!"]

# Get embeddings of sentences
catEmbeddings = model.encode(categories)
sentEmbeddings = model.encode(ourSentences)

In [7]:
# Compute similarities
res = [0]*len(catEmbeddings)
maxSim = -math.inf

# for each category
for i in range(len(catEmbeddings)):
  # calculate the cosine similarity between our sentence embedding 
  # and the given category
  currSim = util.cos_sim(sentEmbeddings, catEmbeddings[i]).item()
  # store the result in a results array
  res[i] = currSim
  # update the maxSim so it stores the highest similarity score
  maxSim = max(maxSim, currSim)


# print("{0:.4f}".format(sim.tolist()[0][0])) 
maxSim

0.2701243460178375

In [8]:
# Now, find the index of the maximum cosine similarity score and retrieve 
# the name of the category
targetCat = categories[res.index(maxSim)]
targetCat

'sympathy'

In [9]:
# Establishing connection for GET request
conn = http.client.HTTPSConnection("famous-quotes4.p.rapidapi.com")
headers = {
    'X-RapidAPI-Key': "12362c4f69mshf3bc1981e616ac1p1939abjsnde94f542c73e",
    'X-RapidAPI-Host': "famous-quotes4.p.rapidapi.com"
    }

# Retrieve 100 quotes with our target category
conn.request("GET", "/random?category={}&count=1000".format(targetCat), headers=headers)

# Retrieving quotes
res = conn.getresponse()
data = res.read()

jsonFormat = json.loads(data)
quotesList = [0]*len(jsonFormat)
for i in range(len(quotesList)):
  quotesList[i] = jsonFormat[i]["text"]

In [10]:
quotesList[0]

"The professional must learn to be moved and touched emotionally, yet at the same time stand back objectively: I've seen a lot of damage done by tea and sympathy."

Part 2: After we got our quotes, we have to repeat the cosine similarity for our sentences and the actual quotes that we retrieved.

In [11]:
quotesEmbeddings = model.encode(quotesList)

In [12]:
quoteRes = [0]*len(quotesEmbeddings)
maxSimRes = -math.inf

# for each category
for i in range(len(quotesEmbeddings)):
  # calculate the cosine similarity between our sentence embedding 
  # and the quote embedding
  currSimRes = util.cos_sim(sentEmbeddings, quotesEmbeddings[i]).item()
  # store the result in a results array
  quoteRes[i] = currSimRes
  # update the maxSimRes so it stores the highest similarity score
  maxSimRes = max(maxSimRes, currSimRes)


# print("{0:.4f}".format(sim.tolist()[0][0])) 
maxSimRes

0.5693548321723938

In [13]:
# Retrieve the quote with the highest similarity score
targetQuote = quotesList[quoteRes.index(maxSimRes)]
targetQuote

'You are 27 or 28 right? It is very tough to live at that age. When nothing is sure. I have sympathy with you.'

Next steps:
1. Figuring out how to retrieve all quotes with a given category tags
2. Amending the code slights to allow for retrieving top-n best matching quotes (linked to "Give me another quote" button)
3. Tuning the model/checking out other pre-defined models OR build/train our own model
4. Exploring other approaches and libraries such as nltk, spacy, etc.